# Vision Library Hardware Emulation with PYNQ

This notebook shows how to set up PYNQ to use the Vitis hardware emulator rather than real hardware. 
The starting point is to build the emulation xclbin file which follows the exact same steps as for 
building one for real hardware.

The only change we need to make is to pass `-t hw_emu` to the `v++` tool.

## Define Kernel code

Write the content of the following cell to the `rgb2hsv.cpp` file. This kernel converts an RGB image to an HSV image

In [ ]:
%%writefile rgb2hsv.cpp

#include "ap_int.h"
#include "common/xf_common.hpp"
#include "common/xf_utility.hpp"
#include "hls_stream.h"
#include "imgproc/xf_rgb2hsv.hpp"

#define AXI_WIDTH 512
#define NPC XF_NPPC8
#define TYPE XF_8UC3
#define MAX_WIDTH 3840
#define MAX_HEIGHT 2160


extern "C"
{
    void rgb2hsv_accel(
        ap_uint<AXI_WIDTH> *image_in,
        ap_uint<AXI_WIDTH> *image_out,
        int width,
        int height)
    {
#pragma HLS INTERFACE m_axi port = image_in offset = slave bundle = image_in_gmem
#pragma HLS INTERFACE m_axi port = image_out offset = slave bundle = image_out_gmem
#pragma HLS INTERFACE s_axilite port = image_in bundle = control
#pragma HLS INTERFACE s_axilite port = image_out bundle = control
#pragma HLS INTERFACE s_axilite port = width bundle = control
#pragma HLS INTERFACE s_axilite port = height bundle = control
#pragma HLS INTERFACE s_axilite port = return bundle = control

        xf::cv::Mat<TYPE, MAX_HEIGHT, MAX_WIDTH, NPC> in_mat(height, width);

        xf::cv::Mat<TYPE, MAX_HEIGHT, MAX_WIDTH, NPC> out_mat(height, width);

#pragma HLS DATAFLOW

        xf::cv::Array2xfMat<AXI_WIDTH, TYPE, MAX_HEIGHT, MAX_WIDTH, NPC>(image_in, in_mat);
        xf::cv::rgb2hsv<TYPE, MAX_HEIGHT, MAX_WIDTH, NPC>(in_mat, out_mat);
        xf::cv::xfMat2Array<AXI_WIDTH, TYPE, MAX_HEIGHT, MAX_WIDTH, NPC>(out_mat, image_out);
    }
}

## Compile the accelerator in Hardware Emulation

First we define the target platform and the path to the Vitis Accelerated libraries 

In [ ]:
import os
home = os.path.expanduser('~') + "/"
platform = home + "aws-fpga/Vitis/aws_platform/xilinx_aws-vu9p-f1_shell-v04261818_201920_2/xilinx_aws-vu9p-f1_shell-v04261818_201920_2.xpfm"
VITIS_LIBS = home + "Vitis_Libraries/vision/L1/include"

Compile the kernel code using `v++`, note that the `-t` switch specifies the compile target and `-k` specifies the kernel to compile. The output is the `rgb2hsv.xo` (Xilinx object) file which contains the compiled kernel.

In [ ]:
!v++ -f $platform -t hw_emu -I$VITIS_LIBS -k rgb2hsv_accel -o rgb2hsv.xo rgb2hsv.cpp

To link the kernel with the rest of the infrastructure in the shell, we use `v++` again but this time with the `-l` switch to generate an `xclbin` file

In [ ]:
!v++ -f $platform -t hw_emu -l -o vision.xclbin rgb2hsv.xo

## Setting up the emulator

With the xclbin file we can start setting up the emulator. First we use the `emconfigutil` utility to define the system we want to emulate. In this case we want a single instance containing the platform we used.

In [ ]:
!emconfigutil -f $platform --nd 1

The `emconfig.json` file describes the platform and is used by the emulator runtime. The default search path for this file is alongside the executable which would be `/usr/bin/python3` for a Python script. We can override this location by setting the `EMCONFIG_PATH` environment variable.

In [ ]:
os.environ['EMCONFIG_PATH'] =  os.environ['PWD']

We also need to set the `XCL_EMULATION_MODE` environment variable to tell PYNQ and the underlying runtime to run against the emulator rather than real hardware. Note that this must be done before importing PYNQ.

In [ ]:
os.environ['XCL_EMULATION_MODE'] = 'hw_emu'

## Running the accelerator in Emulation mode with PYNQ

In [ ]:
import pynq
ol = pynq.Overlay("vision.xclbin")

We can now use PYNQ in exactly the same way as if we were running against real hardware.

Load a test image and compute its size in bytes. It is recommended to use small images to speed up the hardware emulation.

In [ ]:
from PIL import Image
image = Image.open(home + "xup_compute_acceleration/sources/vision_lab/src/data/fish_wallpaper_small.jpg")
img_width, img_height = image.size
size = img_width * img_height * 3

Display original image

In [ ]:
image

Grab a handler to the accelerator and display its signature

In [ ]:
rgb2hsv = ol.rgb2hsv_accel_1
rgb2hsv.signature

Allocate buffers for the input and output images using the `pynq.allocate` API

In [ ]:
import numpy as np
in_buf = pynq.allocate(size, dtype=np.uint8)
out_buf = pynq.allocate(size, dtype=np.uint8)

Move image to the accelerator buffer

In [ ]:
in_buf[:] = np.reshape(np.array(image), size)
in_buf.sync_to_device()

Execute the hardware emulation

In [ ]:
krnl = rgb2hsv.start(in_buf, out_buf, img_width, img_height)

Wait for the emulation to finish and then copy the results

In [ ]:
krnl.wait()
out_buf.sync_from_device()

Display result image

In [ ]:
result_image = np.reshape(out_buf, np.shape(image))
Image.fromarray(result_image, "RGB")

Release the accelerator

In [ ]:
ol.free()
pynq.Device.active_device.close()

----
Copyright © 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause